In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-10-28 03:59:59.071810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 03:59:59.076487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 03:59:59.076801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_process_a", valPreprocessA), 
               ("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-10-28 03:59:59.338671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 03:59:59.339415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 03:59:59.339835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 03:59:59.340137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 227, 227, 3), dtype=float32, numpy=
array([[[[-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         ...,
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ]],

        [[-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         ...,
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ]],

        [[-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         ...,
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241 ],
         [-1.1334219 , -1.3597776 , -1.6293241

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(CROP_HEIGHT, CROP_WIDTH, 3))

    conv1 = tf.keras.layers.Conv2D(96, 7, strides=(4,4), activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv1')
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool1')

    conv2 = tf.keras.layers.Conv2D(256, 5, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv2')
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool2')

    conv3 = tf.keras.layers.Conv2D(384, 3, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv3')
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool3')

    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = conv1(inp)
    o = pool1(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [2, 2], [2, 2], [0, 0]])
    o = conv2(o)
    o = pool2(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [1, 1], [1, 1], [0, 0]])
    o = conv3(o)
    o = pool3(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = flat1(o)
    o = fc1(o)
    o = do1(o)


    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='baseline_1')
    
    return model

### Compile Model

In [28]:
STEPS_PER_EPOCH = 1000

ITERATIONS = 50000*50 // TRAIN_BATCH_SIZE

EPOCH = ITERATIONS // STEPS_PER_EPOCH

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [29]:
def schedule(epoch, lr):
    if(epoch == 10000*50//STEPS_PER_EPOCH):
        return lr * 0.1
    else:
        return lr

In [30]:
logBasePath = "log/baseline_1"
logPrefix = "log"

In [31]:
for i, (train, val) in enumerate(folds):
    
    model = createModel()
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb, lrSchedule]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-10-28 04:00:02.221118: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-10-28 04:00:02.221141: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-10-28 04:00:02.221163: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-10-28 04:00:02.304172: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-10-28 04:00:02.305547: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-10-28 04:00:02.859132: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  51/3528 [..............................] - ETA: 10s - loss: 0.6662 - accuracy: 0.6176  

2022-10-28 04:00:03.633985: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3528/3528 [==============================] - 10s 2ms/step - loss: 0.7207 - accuracy: 0.4768
Epoch 1/625
   1/1000 [..............................] - ETA: 7:51 - loss: 0.7546 - accuracy: 0.5000

2022-10-28 04:00:12.823083: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-10-28 04:00:12.823104: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


...

Epoch 406/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1551 - accuracy: 0.9463 - val_loss: 0.1908 - val_accuracy: 0.9302 - lr: 0.0010
Epoch 407/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1574 - accuracy: 0.9420 - val_loss: 0.2735 - val_accuracy: 0.8970 - lr: 0.0010
Epoch 408/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1752 - accuracy: 0.9388 - val_loss: 0.1318 - val_accuracy: 0.9566 - lr: 0.0010
Epoch 409/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1652 - accuracy: 0.9385 - val_loss: 0.2474 - val_accuracy: 0.9442 - lr: 0.0010
Epoch 410/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1570 - accuracy: 0.9442 - val_loss: 0.4803 - val_accuracy: 0.8216 - lr: 0.0010
Epoch 411/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1517 - accuracy: 0.9490 - val_loss: 0.1641 - val_accuracy: 0.9454 - lr: 0.0010
Epoch 412/625
1000/1000 [===

1000/1000 [==============================] - 13s 13ms/step - loss: 0.1450 - accuracy: 0.9498 - val_loss: 0.2480 - val_accuracy: 0.9072 - lr: 0.0010
Epoch 457/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1509 - accuracy: 0.9475 - val_loss: 0.1429 - val_accuracy: 0.9503 - lr: 0.0010
Epoch 458/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1898 - accuracy: 0.9375 - val_loss: 0.2023 - val_accuracy: 0.9203 - lr: 0.0010
Epoch 459/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1288 - accuracy: 0.9582 - val_loss: 0.3774 - val_accuracy: 0.8579 - lr: 0.0010
Epoch 460/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1522 - accuracy: 0.9457 - val_loss: 0.1496 - val_accuracy: 0.9466 - lr: 0.0010
Epoch 461/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1680 - accuracy: 0.9385 - val_loss: 0.1724 - val_accuracy: 0.9341 - lr: 0.0010
Epoch 462/625
1000/1000 [=================

1000/1000 [==============================] - 13s 13ms/step - loss: 0.1334 - accuracy: 0.9532 - val_loss: 0.1359 - val_accuracy: 0.9508 - lr: 1.0000e-04
Epoch 507/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1205 - accuracy: 0.9560 - val_loss: 0.1302 - val_accuracy: 0.9526 - lr: 1.0000e-04
Epoch 508/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1545 - accuracy: 0.9417 - val_loss: 0.1009 - val_accuracy: 0.9661 - lr: 1.0000e-04
Epoch 509/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1147 - accuracy: 0.9640 - val_loss: 0.1334 - val_accuracy: 0.9512 - lr: 1.0000e-04
Epoch 510/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1155 - accuracy: 0.9615 - val_loss: 0.1090 - val_accuracy: 0.9613 - lr: 1.0000e-04
Epoch 511/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1314 - accuracy: 0.9535 - val_loss: 0.1120 - val_accuracy: 0.9597 - lr: 1.0000e-04
Epoch 512/625
1000

1000/1000 [==============================] - 13s 13ms/step - loss: 0.0811 - accuracy: 0.9735 - val_loss: 0.0702 - val_accuracy: 0.9756 - lr: 1.0000e-04
Epoch 556/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0782 - accuracy: 0.9745 - val_loss: 0.0832 - val_accuracy: 0.9711 - lr: 1.0000e-04
Epoch 557/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0863 - accuracy: 0.9680 - val_loss: 0.0882 - val_accuracy: 0.9678 - lr: 1.0000e-04
Epoch 558/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0830 - accuracy: 0.9725 - val_loss: 0.0730 - val_accuracy: 0.9745 - lr: 1.0000e-04
Epoch 559/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0738 - accuracy: 0.9768 - val_loss: 0.0809 - val_accuracy: 0.9717 - lr: 1.0000e-04
Epoch 560/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0745 - accuracy: 0.9755 - val_loss: 0.0754 - val_accuracy: 0.9739 - lr: 1.0000e-04
Epoch 561/625
1000

1000/1000 [==============================] - 13s 13ms/step - loss: 0.0619 - accuracy: 0.9810 - val_loss: 0.0594 - val_accuracy: 0.9802 - lr: 1.0000e-04
Epoch 605/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0597 - accuracy: 0.9808 - val_loss: 0.0572 - val_accuracy: 0.9810 - lr: 1.0000e-04
Epoch 606/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0762 - accuracy: 0.9745 - val_loss: 0.0579 - val_accuracy: 0.9811 - lr: 1.0000e-04
Epoch 607/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0741 - accuracy: 0.9732 - val_loss: 0.0437 - val_accuracy: 0.9865 - lr: 1.0000e-04
Epoch 608/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0666 - accuracy: 0.9787 - val_loss: 0.0599 - val_accuracy: 0.9802 - lr: 1.0000e-04
Epoch 609/625
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0592 - accuracy: 0.9793 - val_loss: 0.0568 - val_accuracy: 0.9811 - lr: 1.0000e-04
Epoch 610/625
1000